In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import scipy.signal
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

/Users/hn/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')

import remote_sensing_core as rc
import remote_sensing_plot_core as rcp

SF_year = 2016
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/01_NDVI_TS/"
f_name =   "Eastern_WA_" + str(SF_year) + "_70cloud_selectors_max.csv"

In [14]:
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

In [15]:
a_df.head(2)

,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,NDWI,Notes,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8
0,75228_WSDA_SF_2016,23,0.164733,Franklin,Other,developed,wsda,260.0,0.852634,23.202388,...,0.104236,houses,0.228398,NaN,93896.733256,1587.732303,1.442603e+12,T09R29E8,2015.0,0.4021
1,75542_WSDA_SF_2016,21,0.160441,Franklin,Other,developed,wsda,260.0,0.891911,21.213873,...,0.153471,houses,0.228686,NaN,85849.498538,1173.858705,1.442603e+12,T09R29E8,2015.0,0.5054


In [16]:
a_df.system_start_time[1]

1442602882458.0

In [ ]:
epoch_vec = pd.to_datetime(humantimeDF['human_system_start_time']).values.astype(np.int64) // 10 ** 6

In [ ]:
def add_human_start_time(HDF):
    HDF.system_start_time = HDF.system_start_time / 1000
    time_array = HDF["system_start_time"].values.copy()
    human_time_array = [time.strftime('%Y-%m-%d', time.localtime(x)) for x in time_array]
    HDF["human_system_start_time"] = human_time_array
    return(HDF)


In [29]:
HDF = a_df.copy()
HDF = HDF[HDF['system_start_time'].notna()]

In [30]:
HDF.system_start_time = HDF.system_start_time / 1000
time_array = HDF["system_start_time"].values.copy()
human_time_array = [time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)) for x in time_array]
human_time_array_2 = [time.strftime('%Y-%m-%d', time.localtime(x)) for x in time_array]
HDF["human_system_start_time"] = human_time_array
HDF["human_system_start_time_2"] = human_time_array_2

In [31]:
HDF.head(2)

,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8,human_system_start_time,human_system_start_time_2
0,75228_WSDA_SF_2016,23,0.164733,Franklin,Other,developed,wsda,260.0,0.852634,23.202388,...,0.228398,NaN,93896.733256,1587.732303,1.442603e+09,T09R29E8,2015.0,0.4021,2015-09-18 12:01:22,2015-09-18
1,75542_WSDA_SF_2016,21,0.160441,Franklin,Other,developed,wsda,260.0,0.891911,21.213873,...,0.228686,NaN,85849.498538,1173.858705,1.442603e+09,T09R29E8,2015.0,0.5054,2015-09-18 12:01:22,2015-09-18


In [39]:
HDF.iloc[500:550]

,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8,human_system_start_time,human_system_start_time_2
654,8775_WSDA_SF_2016,1,0.026634,Benton,Other,developed,wsda,260.0,0.891884,1.085196,...,0.052493,Grass Hay,4391.633606,269.817103,1.442603e+09,T09R27E19,2015.0,0.4370,2015-09-18 12:01:22,2015-09-18
657,69354_WSDA_SF_2016,11,0.090202,Franklin,Other,developed,wsda,260.0,0.306596,10.544344,...,0.151850,Apple,42671.446610,923.775398,1.442603e+09,T09R30E6,2015.0,0.4992,2015-09-18 12:01:22,2015-09-18
659,9794_WSDA_SF_2016,3,0.061895,Benton,Other,developed,wsda,260.0,0.365679,2.812793,...,0.136961,Alfalfa Hay,11382.970280,892.316925,1.442603e+09,T09R25E23,2015.0,0.3146,2015-09-18 12:01:22,2015-09-18
660,50123_WSDA_SF_2016,1,0.032346,Yakima,Other,developed,wsda,260.0,0.563783,0.580490,...,0.113040,"Corn, Field",2349.158811,217.658711,1.442603e+09,T10R23E24,2015.0,0.4186,2015-09-18 12:01:22,2015-09-18
663,970_WSDA_SF_2016,24,0.131900,Benton,Other,developed,wsda,260.0,0.853232,24.323205,...,0.257661,Alfalfa Hay,98432.519596,1338.569466,1.442603e+09,T08R28E12,2015.0,0.4015,2015-09-18 12:01:22,2015-09-18
664,1047_WSDA_SF_2016,1,0.023589,Benton,Other,developed,wsda,260.0,0.746375,1.211249,...,0.117647,Pasture,4901.750562,281.202013,1.442603e+09,T08R28E16,2015.0,0.3741,2015-09-18 12:01:22,2015-09-18
665,2801_WSDA_SF_2016,14,0.129902,Benton,Other,developed,wsda,260.0,0.646235,13.553750,...,0.219922,Cherry,54850.080323,1001.173259,1.442603e+09,T08R29E14,2015.0,0.3366,2015-09-18 12:01:22,2015-09-18
666,10655_WSDA_SF_2016,3,0.114605,Benton,Other,developed,wsda,260.0,0.351896,3.377231,...,0.160018,Apple,13667.168993,909.541360,1.442603e+09,T08R29E17,2015.0,0.2612,2015-09-18 12:01:22,2015-09-18
667,39686_WSDA_SF_2016,1,0.147366,Yakima,Orchard,cherry,wsda,260.0,0.352178,0.797613,...,0.199181,NaN,3227.824424,337.602081,1.442603e+09,T08R18E25,2015.0,0.2932,2015-09-18 12:01:22,2015-09-18
668,49700_WSDA_SF_2016,73,0.179709,Yakima,Other,pasture,wsda,260.0,0.870399,73.243443,...,0.181615,NaN,296405.696531,3686.745801,1.442603e+09,T08R19E5,2015.0,0.4356,2015-09-18 12:01:22,2015-09-18
